# Load the model for checkpoint

In [1]:
import tensorflow as tf
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [2]:
Config_Path = "D:\\CODE_Club\\Custom_ObjD_2\\ProjectFiles\\MyModel\\SSD\\pipeline.config"
CheckPoint_Path = "D:\\CODE_Club\\Custom_ObjD_2\\ProjectFiles\\MyModel\\SSD"
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(Config_Path)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CheckPoint_Path, 'ckpt-36')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [3]:
category_index = label_map_util.create_category_index_from_labelmap(("D:\\CODE_Club\\Custom_ObjD_2\\ProjectFiles\\Images\\labelmap.pbtxt"))

In [4]:
import cv2 
import numpy as np

cap = cv2.VideoCapture(0)  # Read the video
cap.set(3, 640)  # To set the width of video window to 640px
cap.set(4, 640)  # To set the height of video window to 640px
# The id for brigtness is 10 in cap.set() function



while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=8,
                min_score_thresh=.7,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (640, 640)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break